<a href="https://colab.research.google.com/github/dipakamiitk/Crosslingual-WSD/blob/master/ELECTRA_cls_translate_test_crosslingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
! ls "/content/drive/My Drive/datasets/Split_WiC_dataset"

Cosine_Sim  dev  train


In [ ]:
train_mcl_df = pd.read_csv("/content/drive/MyDrive/datasets/MCL_WiC_old_format/mcl_train_translated_7by8.csv").rename(columns = {"sent2":"byee", "sent2_tr":"sent2"})

train_mcl_df = pd.read_csv("/content/drive/MyDrive/datasets/final/train_rev_mcl.csv")

print(train_mcl_df)
train_mcl_df.head()

            lemma   pos  ...  tag                    id
0            play  NOUN  ...    F      training.en-en.0
1            play  NOUN  ...    F      training.en-en.1
2          esteem  NOUN  ...    T      training.en-en.2
3          esteem  NOUN  ...    T      training.en-en.3
4          holder  NOUN  ...    T      training.en-en.4
...           ...   ...  ...  ...                   ...
15995  positivity  NOUN  ...    T  training.en-en.15995
15996     backing  NOUN  ...    F  training.en-en.15996
15997     backing  NOUN  ...    T  training.en-en.15997
15998        gird  VERB  ...    F  training.en-en.15998
15999        gird  VERB  ...    T  training.en-en.15999

[16000 rows x 12 columns]


,lemma,pos,position1,start1,end1,position2,start2,end2,sent1,sent2,tag,id
0,play,NOUN,11,69,73,2,10,14,In that context of coordination and integratio...,A musical play on the same subject was also st...,F,training.en-en.0
1,play,NOUN,11,69,73,22,112,116,In that context of coordination and integratio...,"In schools, when water is needed, it is girls ...",F,training.en-en.1
2,esteem,NOUN,7,33,39,19,106,112,We would also like to convey our esteem and co...,"Father Lini said that, because of that, the Un...",T,training.en-en.2
3,esteem,NOUN,7,33,39,4,20,26,We would also like to convey our esteem and co...,This attests to the esteem and trust enjoyed b...,T,training.en-en.3
4,holder,NOUN,12,74,81,6,27,33,This growth is the direct result of the increa...,A person may be either the holder of an option...,T,training.en-en.4


In [ ]:
train_mcl_df = pd.concat([train_mcl_df.iloc[:4000,:], train_mcl_df.iloc[4000:,:].rename(columns = { "byee":"sent2", "sent2":"bye"})], ignore_index=True)
print(train_mcl_df)

                       id  ...                                                bye
0        training.en-en.0  ...                                                NaN
1        training.en-en.1  ...                                                NaN
2        training.en-en.2  ...                                                NaN
3        training.en-en.3  ...                                                NaN
4        training.en-en.4  ...                                                NaN
...                   ...  ...                                                ...
7995  training.en-en.7995  ...  Some Instagram users use the app to spread pos...
7996  training.en-en.7996  ...  Between the 1959–1960 overdubs produced by Jac...
7997  training.en-en.7997  ...  It is suggested that people who have pacemaker...
7998  training.en-en.7998  ...  In the 1890s, Confederate memories no longer d...
7999  training.en-en.7999  ...  The eastern side of Sgòr Gaoith is girded by s...

[8000 rows x 15

In [ ]:
train__mcl_rev_df = train_mcl_df.copy().rename(columns = {"sent1":"sent2", "sent2":"sent1", 'start1':'start2', 'start2':'start1', 'end1':'end2', 'end2':'end1'})
train_mcl_df = pd.concat([train_mcl_df, train__mcl_rev_df], ignore_index=True)

In [ ]:
# combine the two datasets
train_df = pd.concat([train_mcl_df], ignore_index=True)
train_df = train_df[~train_df.lemma.str.contains("_")].reset_index()
print(train_df)
train_df.head()
#IDS ARE (NOT (NOT OK)) HUIHUI

       index       lemma  ... tag                    id
0          0        play  ...   F      training.en-en.0
1          1        play  ...   F      training.en-en.1
2          2      esteem  ...   T      training.en-en.2
3          3      esteem  ...   T      training.en-en.3
4          4      holder  ...   T      training.en-en.4
...      ...         ...  ...  ..                   ...
15995  15995  positivity  ...   T  training.en-en.15995
15996  15996     backing  ...   F  training.en-en.15996
15997  15997     backing  ...   T  training.en-en.15997
15998  15998        gird  ...   F  training.en-en.15998
15999  15999        gird  ...   T  training.en-en.15999

[16000 rows x 13 columns]


,index,lemma,pos,position1,start1,end1,position2,start2,end2,sent1,sent2,tag,id
0,0,play,NOUN,11,69,73,2,10,14,In that context of coordination and integratio...,A musical play on the same subject was also st...,F,training.en-en.0
1,1,play,NOUN,11,69,73,22,112,116,In that context of coordination and integratio...,"In schools, when water is needed, it is girls ...",F,training.en-en.1
2,2,esteem,NOUN,7,33,39,19,106,112,We would also like to convey our esteem and co...,"Father Lini said that, because of that, the Un...",T,training.en-en.2
3,3,esteem,NOUN,7,33,39,4,20,26,We would also like to convey our esteem and co...,This attests to the esteem and trust enjoyed b...,T,training.en-en.3
4,4,holder,NOUN,12,74,81,6,27,33,This growth is the direct result of the increa...,A person may be either the holder of an option...,T,training.en-en.4


In [ ]:
dev_df = pd.read_csv("/content/drive/MyDrive/datasets/MCL_WiC_old_format/mcl_dev_translated.csv").rename(columns={"sentence1":"sent1", "sent2_tr":"sent2"})

print(dev_df)
dev_df.head()

In [ ]:
dev_rev_df = dev_df.copy().rename(columns = {"sent1":"sent2", "sent2":"sent1", 'start1':'start2', 'start2':'start1', 'end1':'end2', 'end2':'end1'})
dev_df = pd.concat([dev_df, dev_rev_df], ignore_index=True)
print(dev_df)

                 id  ...                                           sent2_ru
0       dev.en-en.0  ...  Полностью осознавая, что епископы и основные н...
1       dev.en-en.1  ...  В Сенегале сотрудник таможни и его начальство ...
2       dev.en-en.2  ...  Эти жалобы, как правило, подаются на знакомых,...
3       dev.en-en.3  ...  Сексуальное насилие со стороны не-партнеров от...
4       dev.en-en.4  ...  В багаже у меня была венгерская грамматика и с...
...             ...  ...                                                ...
1995  dev.en-en.995  ...  Восемнадцатимесячное французское судебное расс...
1996  dev.en-en.996  ...  Авторы подсчитали, что ежегодная стоимость $ 4...
1997  dev.en-en.997  ...  В докладе подсчитано, что с 1937 года по декаб...
1998  dev.en-en.998  ...  Например, делегат может быть использован для о...
1999  dev.en-en.999  ...  Он пишет, что временная близость этих двух соб...

[2000 rows x 17 columns]


In [ ]:
# test data load chinese
test_df = pd.read_csv("/content/drive/MyDrive/datasets/WiC_dataset/norm_wic_dev_new_format.csv").rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
test_df.head()
print(test_df)



           lemma pos  ...                                              sent2  tag
0          board   N  ...               He nailed boards across the windows.    F
1      circulate   V  ...  This letter is being circulated among the facu...    F
2           hook   V  ...  He hooked a snake accidentally, and was so sca...    T
3     recreation   N  ...  Drug abuse is often regarded as a form of recr...    T
4    domesticity   N  ...  A royal family living in unpretentious domesti...    F
..           ...  ..  ...                                                ...  ...
633         base   N  ...  Bases include oxides and hydroxides of metals ...    F
634        power   N  ...          The mysterious presence of an evil power.    F
635  portmanteau   N  ...  ` motel ' is a portmanteau word made by combin...    T
636      promise   V  ...                       I promised somebody my time.    T
637       pierce   V  ...                       The path pierced the jungle.    F

[638 rows x 11 

In [ ]:
test_df.shape

(638, 11)

In [ ]:
!nvidia-smi

Fri Jan 29 20:55:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 13.9MB/s 
     |████████████████████████████████| 2.9MB 49.7MB/s 
     |████████████████████████████████| 890kB 53.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=513ce09784e3244bb32682410afebba78ea799553742c8d57e9f52fe2d7ddce0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import ElectraTokenizerFast

# Load the BERT tokenizer.
print('Loading ernie tokenizer...')
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")


Loading ernie tokenizer...


In [ ]:
# sentences is a list 0f str


train_sentences_1 = list(train_df['sent1'])
train_sentences_2 = list(train_df['sent2'])

dev_sentences_1 =  list(dev_df['sent1'])
dev_sentences_2 = list(dev_df['sent2'])

test_sentences_1 = list(test_df['sent1'])
test_sentences_2 = list(test_df['sent2'])



print((train_sentences_2[:10]))
print((train_sentences_1[:10]))
print(dev_sentences_1[:10])
print(dev_sentences_2[:10])
print(test_sentences_1[:10])
print(test_sentences_2[:10])

['A musical play on the same subject was also staged in Kathmandu for three days.', 'In schools, when water is needed, it is girls who are sent to fetch it, taking time away from their studies and play.', 'Father Lini said that, because of that, the United Nations has a very special place in the affections and esteem of the people of Vanuatu.', 'This attests to the esteem and trust enjoyed by your country.', 'A person may be either the holder of an option, being the person entitled to buy or sell; or the writer of the option, being the person required to honour the holder’s right to buy or sell.', 'Over 5,000 now hold legal immigrant documents, which, after five years of annual renewal, entitles the holder to apply for permanent residence.', 'The conclusion of the first reading would make it possible to begin negotiations on a streamlined text.', 'It was precisely that consideration which led the Commission on first reading to introduce a special regime of compulsory arbitration in cas

Adding Signal 2

In [ ]:
lem_train = list(train_df['lemma'])
lem_dev = list(dev_df['lemma'])
lem_test = list(test_df['lemma'])

In [ ]:
train_sentences_2 = list((map(lambda x,y: y+" "+x,lem_train, train_sentences_2)))
dev_sentences_2 = list((map(lambda x,y: y+" "+x,lem_dev, dev_sentences_2)))
test_sentences_2 = list((map(lambda x,y: y+" "+x,lem_test, test_sentences_2)))
print(train_sentences_2[:10])

['A musical play on the same subject was also staged in Kathmandu for three days. play', 'In schools, when water is needed, it is girls who are sent to fetch it, taking time away from their studies and play. play', 'Father Lini said that, because of that, the United Nations has a very special place in the affections and esteem of the people of Vanuatu. esteem', 'This attests to the esteem and trust enjoyed by your country. esteem', 'A person may be either the holder of an option, being the person entitled to buy or sell; or the writer of the option, being the person required to honour the holder’s right to buy or sell. holder', 'Over 5,000 now hold legal immigrant documents, which, after five years of annual renewal, entitles the holder to apply for permanent residence. holder', 'The conclusion of the first reading would make it possible to begin negotiations on a streamlined text. reading', 'It was precisely that consideration which led the Commission on first reading to introduce a s

In [ ]:
encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))

encoded_inputs_train_rev = tokenizer(train_sentences_2, train_sentences_1, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

encoded_inputs_test = tokenizer(test_sentences_1, test_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

print(test_sentences_1[2])
print(test_sentences_2[2])
print((encoded_inputs_test['offset_mapping'][2]))
print((encoded_inputs_test['input_ids'][2]))

151
<class 'torch.Tensor'>
128
<class 'torch.Tensor'>
Hook a fish.
He hooked a snake accidentally, and was so scared he dropped his rod into the water. hook
tensor([[ 0,  0],
        [ 0,  4],
        [ 5,  6],
        [ 7, 11],
        [11, 12],
        [ 0,  0],
        [ 0,  2],
        [ 3,  9],
        [10, 11],
        [12, 17],
        [18, 30],
        [30, 31],
        [32, 35],
        [36, 39],
        [40, 42],
        [43, 49],
        [50, 52],
        [53, 60],
        [61, 64],
        [65, 68],
        [69, 73],
        [74, 77],
        [78, 83],
        [83, 84],
        [85, 89],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0

In [ ]:
# labels = torch.from_numpy(train_gold_df['label'].values)
train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
print(train_labels)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

test_gold_df_tmp = test_df[['tag']].replace({'F' : 0, 'T' : 1})
test_labels = torch.from_numpy(test_gold_df_tmp.values)
print(len(test_labels))

tensor([[0],
        [0],
        [1],
        ...,
        [1],
        [0],
        [1]])
638


In [ ]:
from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_labels)


dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_labels)

test_dataset = TensorDataset(encoded_inputs_test['input_ids'], encoded_inputs_test['token_type_ids'],
                              encoded_inputs_test['attention_mask'], test_labels)


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

**CUTOFF**

In [ ]:
from transformers import ElectraForSequenceClassification, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "google/electra-large-discriminator"
        hidden_states = False
        self.encoder = ElectraForSequenceClassification.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, token_type_ids, attention_mask, labels):
        loss, logits = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, labels = labels)

        return loss, logits

In [ ]:
model_bert = BERTi().to(device)


Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

In [ ]:
# model_bert.load_state_dict(torch.load("/content/drive/MyDrive/Model Analysis/Ausemcor/ernie_large_ausem_small_wic/model_bert_dev_best_dev_X_test_X_mcl_and_wic"))

In [ ]:
optimizer = AdamW(list(model_bert.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )

# scheduler??


In [ ]:
def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_accuracy_avg(preds, labels, m):
    non_thresh_preds = (np.exp(preds[:,1])/(np.exp(preds[:,0])+ np.exp(preds[:,1]))).flatten()
    labels_flat = labels.flatten()

    return np.sum(((non_thresh_preds[:m]+non_thresh_preds[m:])>1).astype(int) == labels_flat) / len(labels_flat)

In [ ]:
def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def flat_accuracy_single_logit_avg(preds, labels, m):
    pred_flat = (1/(1+np.exp(-preds))).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(((pred_flat[:m]+pred_flat[m:])>1).astype(int) == labels_flat) / len(labels_flat)

In [ ]:
def cross_entropy(predictions, targets):
    N = predictions.shape[0]
    predictions = (np.exp(predictions[:,1])/(np.exp(predictions[:,0])+ np.exp(predictions[:,1])))
    ce = -np.sum(targets.flatten() * np.log(predictions) + (1-targets.flatten()) * np.log(1-predictions)) / N
    return ce

def cross_entropy_avg(predictions, targets,m):
    N = predictions.shape[0]
    predictions = (np.exp(predictions[:,1])/(np.exp(predictions[:,0])+ np.exp(predictions[:,1])))
    predictions = (predictions[:m]+predictions[m:])/2
    ce = -np.sum(targets.flatten() * np.log(predictions) + (1-targets.flatten()) * np.log(1-predictions)) / N
    return ce

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
path_this = "/content/drive/MyDrive/SemEval Models b/Cross_Lingual/electra_nsp_mcl_rev_wang"


In [ ]:
accumulation_steps = 1

In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 12
best_val_accuracy_n = 0
best_val_accuracy_avg = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()

    if True:
      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):
          # if step == 0:
          #   break

          # Progress update every 40 batches.
          if step % 100 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          if (step == int(len(train_dataloader)/5) or step == int(2*len(train_dataloader)/5) or step == int(3*len(train_dataloader)/5)
          or step == int(4*len(train_dataloader)/5)):
            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            model_bert.eval()

            # Tracking variables 
            total_eval_accuracy = 0
            total_eval_loss = 0
            nb_eval_steps = 0

            t_labels = []
            t_preds = []

            # Evaluate data for one epoch
            for atch in dev_dataloader:
                
                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using 
                # the `to` method.

                b_input_ids = atch[0].to(device)
                b_token_type_ids = atch[1].to(device)
                b_attention_mask = atch[2].to(device)
                b_labels = atch[3].to(device)
                
                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():        

                    loss, logits = model_bert(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)
                    
                # Accumulate the validation loss.
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.cpu().detach().numpy()
                # print(logits)
                label_ids = b_labels.cpu().numpy()

                t_labels.append(label_ids)
                t_preds.append(logits)
                

            all_dev_labels = np.concatenate(t_labels, axis=0)
            all_dev_logits = np.concatenate(t_preds, axis=0)
            # Report the final accuracy for this validation run.
            # print(all_dev_logits)
            avg_val_accuracy_n = flat_accuracy(all_dev_logits[:1000], all_dev_labels[:1000])
            print("Normal Validation Accuracy: {0:.4f}".format(avg_val_accuracy_n))

            # Calculate the average loss over all of the batches.
            # avg_val_loss = total_eval_loss / len(dev_dataloader)
            print("   Normal Validation Loss: {0:.3f}".format(cross_entropy(all_dev_logits[:1000], all_dev_labels[:1000])))

            avg_val_accuracy_avg = flat_accuracy_avg(all_dev_logits, all_dev_labels[:1000], 1000)
            print("Rev Avg Validation Accuracy: {0:.4f}".format(avg_val_accuracy_avg))

            # Calculate the average loss over all of the batches.
            # avg_val_loss = total_eval_loss / len(dev_dataloader)
            print("   Rev Avg Validation Loss: {0:.3f}".format(cross_entropy_avg(all_dev_logits, all_dev_labels[:1000], 1000)))
            
            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)

            if(avg_val_accuracy_n > best_val_accuracy_n):
              # flag = True
              best_val_accuracy_n = avg_val_accuracy_n
              best_val_preds = all_dev_logits
              np.save(os.path.join(path_this, "dev_preds_normal"), best_val_preds)
              # best_bert_parameters = model_bert.state_dict()
              # best_log_reg_parameters = model_log_reg.state_dict()    
              torch.save(model_bert.state_dict(), os.path.join(path_this, "normal_model_bert_dev_best"))

            if(avg_val_accuracy_avg > best_val_accuracy_avg):
              # flag = True
              best_val_accuracy_avg = avg_val_accuracy_avg
              best_val_preds = all_dev_logits
              np.save(os.path.join(path_this, "rev_avg_dev_preds"), best_val_preds)
              # best_bert_parameters = model_bert.state_dict()
              # best_log_reg_parameters = model_log_reg.state_dict()    
              torch.save(model_bert.state_dict(), os.path.join(path_this, "rev_avg_model_bert_dev_best"))
            
            # print("  Validation Loss: {0:.3f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

            model_bert.train()


          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.

          b_input_ids = batch[0].to(device)
          b_token_type_ids = batch[1].to(device)
          b_attention_mask = batch[2].to(device)
          b_labels = batch[3].to(device)

          if step%accumulation_steps == 0:
            optimizer.zero_grad()        

          loss, logits = model_bert(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)

          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          # TODO logsistic REG????/
          if (step%accumulation_steps == (accumulation_steps-1)):
            torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)
            optimizer.step()

          # # Update the learning rate.
          # scheduler.step()
        
    

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            loss, logits = model_bert(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy_n = flat_accuracy(all_dev_logits[:1000], all_dev_labels[:1000])
    print("Normal Validation Accuracy: {0:.4f}".format(avg_val_accuracy_n))

    # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(dev_dataloader)
    print("   Normal Validation Loss: {0:.3f}".format(cross_entropy(all_dev_logits[:1000], all_dev_labels[:1000])))

    avg_val_accuracy_avg = flat_accuracy_avg(all_dev_logits, all_dev_labels[:1000], 1000)
    print("Rev Avg Validation Accuracy: {0:.4f}".format(avg_val_accuracy_avg))

    # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(dev_dataloader)
    print("   Rev Avg Validation Loss: {0:.3f}".format(cross_entropy_avg(all_dev_logits, all_dev_labels[:1000], 1000)))
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy_n > best_val_accuracy_n):
      # flag = True
      best_val_accuracy_n = avg_val_accuracy_n
      best_val_preds = all_dev_logits
      np.save(os.path.join(path_this, "dev_preds_normal"), best_val_preds)
      # best_bert_parameters = model_bert.state_dict()
      # best_log_reg_parameters = model_log_reg.state_dict()    
      torch.save(model_bert.state_dict(), os.path.join(path_this, "normal_model_bert_dev_best"))

    if(avg_val_accuracy_avg > best_val_accuracy_avg):
      # flag = True
      best_val_accuracy_avg = avg_val_accuracy_avg
      best_val_preds = all_dev_logits
      np.save(os.path.join(path_this, "rev_avg_dev_preds"), best_val_preds)
      # best_bert_parameters = model_bert.state_dict()
      # best_log_reg_parameters = model_log_reg.state_dict()    
      torch.save(model_bert.state_dict(), os.path.join(path_this, "rev_avg_model_bert_dev_best"))
    
    # print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    
    # fgfg

    # # Do same on test data
    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in test_dataloader:
        
    #     # Unpack this training batch from our dataloader. 
    #     #
    #     # As we unpack the batch, we'll also copy each tensor to the GPU using 
    #     # the `to` method.

    #     b_input_ids = batch[0].to(device)
    #     b_token_type_ids = batch[1].to(device)
    #     b_attention_mask = batch[2].to(device)
    #     b_poses = batch[3].to(device)
    #     b_labels = batch[4].to(device)
        
    #     # Tell pytorch not to bother with constructing the compute graph during
    #     # the forward pass, since this is only needed for backprop (training).
    #     with torch.no_grad():
    #         last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 4096)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Test Accuracy: {0:.4f}".format(avg_val_accuracy))

    # if flag == True:
    #   np.save(os.path.join(path_this, "test_preds_at_best_dev"), best_val_preds)
    #   flag = False

    # if(avg_val_accuracy > t_best_val_accuracy):
    #   t_best_val_accuracy = avg_val_accuracy
    #   t_best_val_preds = all_dev_logits
    #   np.save(os.path.join(path_this, "test_preds"), t_best_val_preds)
    #   torch.save(model_bert.state_dict(), os.path.join(path_this, "model_bert_test_best"))
    #   torch.save(model_log_reg.state_dict(), os.path.join(path_this, "model_log_reg_test_best"))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(test_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Test Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Testing Validation took: {:}".format(validation_time))

   
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 12 ========
Training...
  Batch   100  of  1,000.    Elapsed: 0:00:49.
  Batch   200  of  1,000.    Elapsed: 0:01:37.

Running Validation...
Normal Validation Accuracy: 0.6680
   Normal Validation Loss: 0.635
Rev Avg Validation Accuracy: 0.6750
   Rev Avg Validation Loss: 0.306
  Validation took: 0:00:14
  Batch   300  of  1,000.    Elapsed: 0:01:23.
  Batch   400  of  1,000.    Elapsed: 0:02:11.

Running Validation...
Normal Validation Accuracy: 0.7640
   Normal Validation Loss: 0.532
Rev Avg Validation Accuracy: 0.7610
   Rev Avg Validation Loss: 0.266
  Validation took: 0:00:14
  Batch   500  of  1,000.    Elapsed: 0:01:23.
  Batch   600  of  1,000.    Elapsed: 0:02:11.

Running Validation...
Normal Validation Accuracy: 0.8260
   Normal Validation Loss: 0.428
Rev Avg Validation Accuracy: 0.8220
   Rev Avg Validation Loss: 0.213
  Validation took: 0:00:14
  Batch   700  of  1,000.    Elapsed: 0:01:24.
  Batch   800  of  1,000.    Elapsed: 0:02:13.

Running Validat

KeyboardInterrupt: ignored

In [ ]:
print(best_val_accuracy_n) #mcl_rev_wang
print(best_val_accuracy_avg)

0.864
0.855
